In [ ]:
# Workshop Setup
import sys
sys.path.insert(0, '..')

from setup_llm import verify_setup, get_chat_model, get_embeddings

verify_setup()

print("\n📚 Session 2, Module 5: Context Engineering")
print("=" * 60)
print("\n🔗 Prerequisites:")
print("   • Module 1: Memory architecture")
print("   • Module 2: Advanced HITL")
print("   • Module 3: Self-reflection patterns")
print("   • Module 4: Multi-agent collaboration")
print("\n🎯 Optimize agent context for production:")
print("\nPart 1: The Four Pillars (25 min)")
print("  • WRITE - Save outside context (Store, Memory)")
print("  • SELECT - Pull in when needed (RAG, Retrieval)")
print("  • COMPRESS - Keep only relevant (Summarization, Trim)")
print("  • ISOLATE - Split up context (Multi-agent, Subgraphs)")
print("\nPart 2: Production Middleware (15 min)")
print("  • ModelRetryMiddleware for reliability")
print("  • Stacking middleware for production agents")
print("\nPart 3: Memory-Augmented RAG Demo (30 min)")
print("  • Dual retrieval (static + conversational)")
print("  • User profile personalization")
print("  • Production metrics and patterns")

🔍 Checking LLM Configuration...
📡 Provider: DIAL (Azure OpenAI via EPAM AI Proxy)
✅ DIAL_API_KEY is set

📋 Configuration:
   AZURE_OPENAI_ENDPOINT: https://ai-proxy.lab.epam.com
   AZURE_OPENAI_API_VERSION: 2024-08-01-preview
   AZURE_OPENAI_DEPLOYMENT_NAME: gpt-4

✅ DIAL setup verified successfully!

📚 Session 2, Module 5: Context Engineering

🔗 Prerequisites:
   • Module 1: Memory architecture
   • Module 2: Advanced HITL
   • Module 3: Self-reflection patterns
   • Module 4: Multi-agent collaboration

🎯 Optimize agent context for production:

Part 1: The Four Pillars (20 min)
  • WRITE - Save outside context (Store, Memory)
  • SELECT - Pull in when needed (RAG, Retrieval)
  • COMPRESS - Keep only relevant (Summarization, Trim)
  • ISOLATE - Split up context (Multi-agent, Subgraphs)

Part 2: LangChain Middleware (20 min)
  • SummarizationMiddleware for context compression
  • ModelRetryMiddleware for reliability
  • Combining middleware for production

Part 3: Memory-Augmented RAG (

---

## Why Context Engineering Matters

### The Problem

When agents fail in production, **the right context was not passed to the LLM** (not because the model isn't capable).

**Why agents are challenging:**
- **Long-running tasks** → Context accumulates across many turns
- **Tool calling** → Each tool adds feedback to context

**Example**: 10 tool calls × 500 tokens each = 5,000+ tokens just from tool feedback!

### The LLM as an Operating System

- **LLM = CPU**
- **Context Window = RAM**
- **Context Engineering = Memory Management**

Just like an OS decides what fits in RAM, context engineering decides what fits in the context window.

### Context Failures to Avoid

1. **Poisoning** - Incorrect info influences responses
2. **Distraction** - Irrelevant info confuses the LLM
3. **Curation** - Missing critical information
4. **Clash** - Conflicting information

---

## The Four Pillars: WRITE, SELECT, COMPRESS, ISOLATE

```
┌────────────────────────────────────────────────────────────┐
│                                                            │
│  WRITE          SELECT        COMPRESS       ISOLATE      │
│  ───────        ───────       ────────       ────────      │
│  Save outside   Pull in       Keep only      Split up     │
│  context        when needed   relevant       context       │
│                                                            │
│  Examples:      Examples:     Examples:      Examples:     │
│  • Scratchpad   • Memories    • Summarize    • Multi-agent│
│  • Memory       • Tools (RAG) • Trim         • Sandbox     │
│  • Files        • Knowledge   • Post-process • State fields│
│                                                            │
└────────────────────────────────────────────────────────────┘
```

*Framework from: Claude Code, Cursor, Windsurf, Anthropic's Multi-Agent Researcher, Cognition's Devin*

---

## Pillar 1: WRITE — Save Outside Context

**Goal**: Write information to persistent storage so it doesn't consume context window space.

### The Concept

- **Scratchpad (RAM)**: Short-term, in context, for current task
- **Memory (Disk)**: Long-term, external storage, retrieved when needed

### Three Types of Memory

1. **Semantic** - Facts (User's API keys in AWS Secrets Manager)
2. **Episodic** - Events (Nov 25: user reported bug in module X)
3. **Procedural** - How-tos (To deploy: `make deploy && kubectl rollout`)

In [3]:
# WRITE Pillar Example: LangGraph Store
from langgraph.store.memory import InMemoryStore

print("📝 WRITE Pillar: Using LangGraph Store")
print("=" * 50)

# Initialize store
store = InMemoryStore()

# Save fact outside context window
store.put(
    namespace=("user_prefs", "user_123"),
    key="theme",
    value={"preference": "dark_mode", "font_size": 14}
)

# Save procedural memory
store.put(
    namespace=("procedures", "deployment"),
    key="steps",
    value={"command": "make deploy && kubectl rollout", "environment": "production"}
)

print("\n✅ Saved to external store (outside context window)")

# Retrieve when needed
prefs = store.search(("user_prefs", "user_123"))
print(f"\n📖 Retrieved preferences: {list(prefs)}")

print("\n💡 Key Difference:")
print("  • State = Current context (in window)")
print("  • Store = Long-term knowledge (outside window)")

📝 WRITE Pillar: Using LangGraph Store

✅ Saved to external store (outside context window)

📖 Retrieved preferences: [Item(namespace=['user_prefs', 'user_123'], key='theme', value={'preference': 'dark_mode', 'font_size': 14}, created_at='2025-12-05T05:58:13.320721+00:00', updated_at='2025-12-05T05:58:13.320728+00:00', score=None)]

💡 Key Difference:
  • State = Current context (in window)
  • Store = Long-term knowledge (outside window)


---

## Pillar 2: SELECT — Pull Context When Needed

**Goal**: Dynamically retrieve only relevant information into context.

### The Challenge

You have 100+ tools or 1000+ documents. You can't fit them all in context!

### Solution: Retrieval Augmented Generation (RAG)

#### 2A: Tool Selection via RAG
When you have 30+ tools, embed descriptions and retrieve the top-K relevant ones.

#### 2B: Memory Retrieval
Retrieve relevant past conversations/facts.

#### 2C: Knowledge Retrieval (Traditional RAG)
Retrieve from document knowledge base.

In [5]:
# SELECT Pillar Example: Tool Selection via RAG
from langchain_community.vectorstores import FAISS

print("🔍 SELECT Pillar: Dynamic Tool Selection")
print("=" * 50)

# Tool descriptions (imagine you have 30+ tools)
tool_descriptions = [
    "search_web: Search the internet for current information",
    "calculate: Perform mathematical calculations",
    "send_email: Send an email to a recipient",
    "read_file: Read contents of a file",
    "write_file: Write content to a file",
    "search_database: Query the SQL database",
    "get_weather: Get current weather for a location",
    "translate: Translate text between languages",
]

# Create embeddings and vector store
embeddings = get_embeddings()
tool_vectorstore = FAISS.from_texts(tool_descriptions, embeddings)

def select_tools(query: str, k: int = 3):
    """SELECT pattern: Retrieve top-k relevant tools for query"""
    relevant_tools = tool_vectorstore.similarity_search(query, k=k)
    return [doc.page_content for doc in relevant_tools]

# Example usage
query = "I need to send a message to my colleague"
selected = select_tools(query)
print(f"\n📝 Query: '{query}'")
print(f"\n🎯 Selected tools (top 3):")
for i, tool in enumerate(selected, 1):
    print(f"   {i}. {tool}")

print("\n💡 Used by: Windsurf, Cursor (when tool count is high)")

🔍 SELECT Pillar: Dynamic Tool Selection

📝 Query: 'I need to send a message to my colleague'

🎯 Selected tools (top 3):
   1. send_email: Send an email to a recipient
   2. translate: Translate text between languages
   3. search_web: Search the internet for current information

💡 Used by: Windsurf, Cursor (when tool count is high)


---

## Pillar 3: COMPRESS — Keep Only Relevant Tokens

**Goal**: Reduce context size by summarizing or trimming.

### When to COMPRESS

- Long conversation histories
- Verbose tool outputs
- Repeated information

### Patterns

- **3A: Summarization** - Summarize old messages
- **3B: Trim by Recency** - Keep only recent messages
- **3C: Tool Output Filtering** - Truncate verbose tool outputs

In [6]:
# COMPRESS Pillar Example: Message Summarization
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from typing import TypedDict

print("🗜️ COMPRESS Pillar: Message Summarization")
print("=" * 50)

class AgentState(TypedDict):
    messages: list

def compress_conversation(state: AgentState) -> dict:
    """COMPRESS pattern: Summarize old messages when context grows"""
    messages = state["messages"]
    
    if len(messages) > 10:
        # Get LLM to summarize older messages
        old_messages = messages[:7]
        recent_messages = messages[7:]
        
        # Create summary prompt
        summary_prompt = "Summarize the key points from this conversation:"
        
        llm = get_chat_model()
        summary = llm.invoke([
            SystemMessage(content=summary_prompt),
            *old_messages
        ])
        
        # Replace old messages with summary
        compressed = [
            SystemMessage(content=f"Previous conversation summary: {summary.content}"),
            *recent_messages
        ]
        
        print(f"\n📊 Compression Results:")
        print(f"   Before: {len(messages)} messages")
        print(f"   After: {len(compressed)} messages")
        
        return {"messages": compressed}
    
    return {}

print("\n✅ Compression function defined")

print("\n💡 Tool Output Filtering Example:")
print("""
def filter_tool_output(tool_name: str, result: str) -> str:
    if tool_name == "search_web":
        return result[:500] + "..." if len(result) > 500 else result
    elif tool_name == "read_file":
        return result[:1000] + "\n... (truncated)" if len(result) > 1000 else result
    return result
""")

🗜️ COMPRESS Pillar: Message Summarization

✅ Compression function defined

💡 Tool Output Filtering Example:

def filter_tool_output(tool_name: str, result: str) -> str:
    if tool_name == "search_web":
        return result[:500] + "..." if len(result) > 500 else result
    elif tool_name == "read_file":
        return result[:1000] + "
... (truncated)" if len(result) > 1000 else result
    return result



### Production COMPRESS: SummarizationMiddleware

LangChain's `SummarizationMiddleware` automates context compression for production:

| Trigger Type | Example | When to Use |
|------------|---------|-------------|
| `("messages", N)` | Trigger at N messages | Simple message-count threshold |
| `("tokens", N)` | Trigger at N tokens | Token-aware compression |
| `("fraction", 0.8)` | Trigger at 80% context | Model-aware, recommended |

> **Note**: Other middleware (retry, fallback, HITL) are covered in Part 2 after we complete the Four Pillars.

In [ ]:
# SummarizationMiddleware - Automatic Conversation Compression
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langchain_core.tools import tool

# Initialize model
model = get_chat_model(temperature=0)

@tool
def search_docs(query: str) -> str:
    """Search internal documentation."""
    return f"Found 3 documents about '{query}': Architecture guide, API reference, Deployment docs..."

@tool
def get_status(service: str) -> str:
    """Get service status."""
    return f"Service '{service}' is running. Uptime: 99.9%, Latency: 45ms"

# Create agent with SummarizationMiddleware (COMPRESS pillar)
# NOTE: trigger and keep use TUPLE format: ("messages", N) not dict format
agent_with_summarization = create_agent(
    model=model,
    tools=[search_docs, get_status],
    middleware=[
        SummarizationMiddleware(
            model=model,                      # Use same model (or cheaper one for summarization)
            trigger=("messages", 10),         # Trigger when >10 messages (TUPLE format!)
            keep=("messages", 5),             # Keep 5 most recent messages (TUPLE format!)
        ),
    ]
)

print("✅ SummarizationMiddleware configured (COMPRESS pillar)")
print("""
How it works:
  1. Agent accumulates messages during conversation
  2. When trigger threshold is reached (10 messages):
     - Old messages are summarized into a single message
     - Recent messages (5) are kept verbatim
  3. Context stays manageable for long conversations

This is the COMPRESS pillar in action!
""")

✅ SummarizationMiddleware configured

How it works:
  1. Agent accumulates messages during conversation
  2. When trigger threshold is reached (10 messages):
     - Old messages are summarized into a single message
     - Recent messages (5) are kept verbatim
  3. Context stays manageable for long conversations

Trigger Options (use TUPLE format!):
  • ("messages", N)  - Count of messages
  • ("tokens", N)    - Token count
  • ("fraction", 0.8) - Fraction of model's context window
  • Combine: [("tokens", 3000), ("messages", 6)] (OR logic)



---

## Pillar 4: ISOLATE — Split Up Context

**Goal**: Separate concerns by spawning specialized subagents or isolating state.

### Patterns

- **4A: Multi-Agent (Supervisor)** - Each subagent has isolated context
- **4B: State Schema Isolation** - Use Pydantic models to separate concerns
- **4C: Code Execution Sandboxing** - Run untrusted code in isolated environment

**Benefits**:
- Researcher doesn't see writer's context
- Prevents context clash
- Each agent optimized for its task

In [9]:
# ISOLATE Pillar Example: Multi-Agent Isolation
from typing import TypedDict
from pydantic import BaseModel, Field

print("🔒 ISOLATE Pillar: Multi-Agent Isolation")
print("=" * 50)

# Different state schemas for different agents
class SupervisorState(TypedDict):
    task: str
    result: str

class ResearcherState(TypedDict):
    query: str
    findings: str

class WriterState(TypedDict):
    draft: str
    feedback: str

def supervisor(state: SupervisorState) -> str:
    """Supervisor routes to subagents with isolated context"""
    if "research" in state["task"]:
        return "researcher"  # Isolated context
    return "writer"  # Different isolated context

print("\n✅ Multi-Agent State Isolation defined")

print("\n💡 State Schema Isolation with Pydantic:")

class AgentStateWithIsolation(BaseModel):
    """State with isolated fields per agent"""
    # Public context (available to all nodes)
    messages: list = Field(default_factory=list)
    
    # Private context (isolated per node)
    researcher_context: str | None = Field(
        default=None,
        description="Only for researcher node"
    )
    writer_context: str | None = Field(
        default=None,
        description="Only for writer node"
    )
    
print("\n✅ Pydantic state with isolation defined")
print("\n🎯 Benefits of ISOLATE:")
print("  • Researcher doesn't see writer's context")
print("  • Prevents context clash between subtasks")
print("  • Each agent optimized for its specific task")

🔒 ISOLATE Pillar: Multi-Agent Isolation

✅ Multi-Agent State Isolation defined

💡 State Schema Isolation with Pydantic:

✅ Pydantic state with isolation defined

🎯 Benefits of ISOLATE:
  • Researcher doesn't see writer's context
  • Prevents context clash between subtasks
  • Each agent optimized for its specific task


---

## LangGraph Features Mapped to Pillars

| Pillar | LangGraph Feature | Purpose |
|--------|------------------|--------|
| WRITE | `Store` | Save memories outside state |
| WRITE | State fields | Scratchpad within session |
| SELECT | `Store.search()` | Retrieve relevant memories |
| SELECT | Tool binding | Dynamically select tools |
| COMPRESS | Reducers | Summarize/trim state fields |
| COMPRESS | Middleware | Post-process tool outputs |
| ISOLATE | Subgraphs | Spawn isolated subagents |
| ISOLATE | State schema | Separate concerns via types |

---

## Part 2: Production Middleware

Now that we understand the Four Pillars conceptually, let's look at **LangChain middleware** that helps implement them in production.

### Middleware Overview

| Middleware | Pillar | Purpose | When to Use |
|------------|--------|---------|-------------|
| `SummarizationMiddleware` | COMPRESS | Condense conversation history | Long conversations |
| `ModelRetryMiddleware` | Reliability | Auto-retry on failures | Production systems |
| `ModelFallbackMiddleware` | Reliability | Fallback to alternative models | High availability |
| `HumanInTheLoopMiddleware` | Safety | Require human approval | Sensitive operations |
| `PIIMiddleware` | Privacy | Redact sensitive data | User-facing apps |

> **Note**: We already saw `SummarizationMiddleware` under COMPRESS. Below are the remaining middleware patterns.

In [ ]:
# ModelRetryMiddleware - Production Reliability
from langchain.agents.middleware import ModelRetryMiddleware

# Agent with automatic retry on failures
agent_with_retry = create_agent(
    model=model,
    tools=[search_docs, get_status],
    middleware=[
        ModelRetryMiddleware(
            max_retries=3,           # Maximum retry attempts
            backoff_factor=2.0,      # Exponential backoff multiplier
            initial_delay=1.0,       # Initial delay in seconds
            # Retry sequence: 1s → 2s → 4s
        ),
    ]
)

print("✅ ModelRetryMiddleware configured")
print("""
How it works:
  1. Model call fails (rate limit, timeout, etc.)
  2. Wait initial_delay seconds (1s)
  3. Retry, wait initial_delay * backoff_factor (2s)
  4. Retry, wait previous * backoff_factor (4s)
  5. After max_retries, raise exception

Best Practices:
  • Set max_retries=3 for balance between reliability and latency
  • Use backoff_factor=2.0 for exponential backoff
  • Combine with ModelFallbackMiddleware for high availability
""")

In [ ]:
# Combining Multiple Middleware - Production Best Practice
from langchain.agents.middleware import (
    SummarizationMiddleware,
    ModelRetryMiddleware,
    HumanInTheLoopMiddleware
)
from langgraph.checkpoint.memory import InMemorySaver

# Production-ready agent with stacked middleware
production_agent = create_agent(
    model=model,
    tools=[search_docs, get_status],
    middleware=[
        # Layer 1: Retry on transient failures (Reliability)
        ModelRetryMiddleware(
            max_retries=3,
            backoff_factor=2.0,
        ),
        # Layer 2: Compress long conversations (COMPRESS pillar)
        SummarizationMiddleware(
            model=model,
            trigger=("messages", 15),
            keep=("messages", 5),
        ),
        # Layer 3: Human approval for sensitive operations (Safety)
        HumanInTheLoopMiddleware(
            interrupt_on={"get_status": False}  # Customize per tool
        ),
    ],
    checkpointer=InMemorySaver()  # Required for HITL
)

print("✅ Production agent with stacked middleware")
print("""
Middleware Execution Order (stack):
  Request → ModelRetry → Summarization → HITL → Agent
  Response ← ModelRetry ← Summarization ← HITL ← Agent

Middleware mapped to concepts:
  • ModelRetryMiddleware → Reliability (not a pillar, but essential)
  • SummarizationMiddleware → COMPRESS pillar
  • HumanInTheLoopMiddleware → Safety/HITL (Module 2 topic)

This is the recommended pattern for production agents!
""")

### Middleware Summary

| Middleware | Category | Key Parameters | Use Case |
|------------|----------|----------------|----------|
| `SummarizationMiddleware` | COMPRESS | `trigger`, `keep`, `model` | Long conversations |
| `ModelRetryMiddleware` | Reliability | `max_retries`, `backoff_factor` | Production systems |
| `HumanInTheLoopMiddleware` | Safety | `interrupt_on` | Sensitive operations |
| `ModelFallbackMiddleware` | Reliability | `fallback_models` | High availability |
| `PIIMiddleware` | Privacy | `strategy`, `apply_to_input` | User-facing apps |
| `ToolRetryMiddleware` | Reliability | `max_retries` | Unreliable external APIs |

**Production Baseline**: Stack `ModelRetryMiddleware` + `SummarizationMiddleware` as minimum.

---

## Part 3: Memory-Augmented RAG Demo

Now let's put the Four Pillars into practice with a real-world demo.

In [ ]:
# Demo: Personalized Document Assistant with Memory-Augmented RAG

import os
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, START, END
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from datetime import datetime

print("""
╔═══════════════════════════════════════════════════════════════╗
║                                                               ║
║         PERSONALIZED DOCUMENT ASSISTANT                       ║
║         Memory-Augmented RAG (Notion AI Style)                ║
║                                                               ║
║         Query → Dual Retrieval → Generate → Update Memory     ║
║                                                               ║
╚═══════════════════════════════════════════════════════════════╝

🎯 BUSINESS USE CASE: Enterprise Knowledge Management

📊 REAL-WORLD METRICS:
   • 30M+ users (Notion AI scale)
   • 45% relevance improvement vs traditional RAG
   • 60% reduction in repeated questions
   • 4.5/5 user satisfaction (up from 3.2)
""")

In [ ]:
# Enterprise Documents (Sample Knowledge Base)

ENTERPRISE_DOCUMENTS = [
    # HR Policies
    {
        "title": "Remote Work Policy 2024",
        "content": """
        Our company supports flexible remote work arrangements. Employees may work remotely up to 3 days per week.
        Remote work requires manager approval and must maintain productivity standards.
        Equipment reimbursement: $500/year for home office setup.
        Communication: Daily check-ins via Slack, weekly team video calls required.
        """,
        "category": "HR",
        "department": "Human Resources"
    },
    {
        "title": "PTO and Vacation Policy",
        "content": """
        Employees receive 20 days of PTO annually, accrued monthly (1.67 days/month).
        PTO requests require 2 weeks notice for trips >5 days.
        Unused PTO can roll over up to 5 days to the next year.
        Sick leave is unlimited but requires documentation for absences >3 consecutive days.
        """,
        "category": "HR",
        "department": "Human Resources"
    },
    # Engineering Docs
    {
        "title": "API Authentication Guide",
        "content": """
        All API requests must include a Bearer token in the Authorization header.
        Tokens expire after 24 hours and must be refreshed using the /auth/refresh endpoint.
        Rate limits: 1000 requests/hour for standard tier, 10000/hour for enterprise.
        Use API keys (not tokens) for server-to-server communication.
        """,
        "category": "Engineering",
        "department": "Engineering"
    },
    {
        "title": "Deployment Process",
        "content": """
        All code must pass CI/CD checks before merging to main branch.
        Deployments to production happen every Tuesday and Thursday at 2pm PST.
        Emergency hotfixes require approval from two senior engineers.
        Rollback procedure: Run './scripts/rollback.sh <version>' from ops server.
        """,
        "category": "Engineering",
        "department": "Engineering"
    },
    # Sales Materials
    {
        "title": "Q4 2024 Pricing",
        "content": """
        Starter Plan: $99/month, up to 10 users, basic features.
        Professional Plan: $299/month, up to 50 users, advanced analytics.
        Enterprise Plan: Custom pricing, unlimited users, dedicated support, SLA.
        Annual contracts receive 20% discount.
        """,
        "category": "Sales",
        "department": "Sales"
    }
]

print(f"✅ Loaded {len(ENTERPRISE_DOCUMENTS)} enterprise documents")

In [ ]:
# State Definition

class MemoryRAGState(TypedDict):
    """State for memory-augmented RAG system"""
    
    # Input
    user_id: str
    question: str
    
    # Memory (Dual Source)
    conversation_history: list  # Short-term: Recent Q&A pairs
    user_profile: dict  # Long-term: Department, role, preferences
    learned_preferences: dict  # Semantic: Learned over time
    
    # Retrieval
    retrieved_static_docs: list  # From knowledge base
    retrieved_memory_context: list  # From conversation history
    relevant_user_facts: list  # From user profile
    
    # Generation
    answer: str
    sources_used: list
    
    # Memory Update
    extracted_facts: dict  # New facts to store
    memory_updates: dict

print("✅ MemoryRAGState defined")

In [ ]:
# Initialize Vector Store and Helper Functions

def initialize_vector_store():
    """Initialize FAISS vector store with enterprise documents."""
    documents = []
    for doc in ENTERPRISE_DOCUMENTS:
        documents.append(Document(
            page_content=doc["content"],
            metadata={
                "title": doc["title"],
                "category": doc["category"],
                "department": doc["department"]
            }
        ))
    
    embeddings = get_embeddings()  # Uses DIAL
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore


def load_user_profile(user_id: str) -> dict:
    """Load user profile from database (simulated)."""
    user_profiles = {
        "user_eng_001": {
            "name": "Alice Chen",
            "department": "Engineering",
            "role": "Senior Backend Engineer",
            "team": "Platform",
            "interests": ["API design", "performance optimization", "deployment"],
            "recent_topics": ["authentication", "rate limiting"]
        },
        "user_hr_001": {
            "name": "Bob Martinez",
            "department": "Human Resources",
            "role": "HR Manager",
            "team": "People Ops",
            "interests": ["policies", "benefits", "remote work"],
            "recent_topics": ["PTO", "vacation policy"]
        },
        "user_sales_001": {
            "name": "Carol Johnson",
            "department": "Sales",
            "role": "Account Executive",
            "team": "Enterprise Sales",
            "interests": ["pricing", "competitor analysis", "objection handling"],
            "recent_topics": ["enterprise pricing", "competitors"]
        }
    }
    
    return user_profiles.get(user_id, {
        "name": "Guest User",
        "department": "General",
        "role": "Employee",
        "interests": [],
        "recent_topics": []
    })


def load_conversation_history(user_id: str) -> list:
    """Load recent conversation history (simulated)."""
    history_db = {
        "user_eng_001": [
            {"question": "How do I authenticate API requests?", "answer": "Use Bearer tokens in Authorization header..."},
            {"question": "What's the rate limit?", "answer": "1000 requests/hour for standard tier..."}
        ],
        "user_hr_001": [
            {"question": "How many PTO days do we get?", "answer": "20 days annually, accrued monthly..."}
        ],
        "user_sales_001": [
            {"question": "What's our pricing for enterprise?", "answer": "Custom pricing for Enterprise plan..."},
            {"question": "Who are our main competitors?", "answer": "CompanyX, CompanyY, and CompanyZ..."}
        ]
    }
    
    return history_db.get(user_id, [])


# Initialize vector store
print("\n🔧 Initializing vector store with enterprise documents...")
vectorstore = initialize_vector_store()
print("✅ Vector store ready!")

In [ ]:
# Load User Context Node

def load_user_context(state: MemoryRAGState) -> dict:
    """
    Load user profile and conversation history.
    
    Context Engineering Pillar: WRITE (to state) + SELECT (from memory)
    """
    
    print(f"\n{'='*70}")
    print(f"👤 LOADING USER CONTEXT: {state['user_id']}")
    print(f"{'='*70}\n")
    
    # Load user profile
    profile = load_user_profile(state["user_id"])
    print(f"📋 User Profile:")
    print(f"   Name: {profile.get('name', 'Unknown')}")
    print(f"   Department: {profile.get('department', 'Unknown')}")
    print(f"   Role: {profile.get('role', 'Unknown')}")
    print(f"   Interests: {', '.join(profile.get('interests', []))}")
    
    # Load conversation history
    history = load_conversation_history(state["user_id"])
    print(f"\n💬 Conversation History: {len(history)} previous interactions")
    
    # Initialize learned preferences
    learned_prefs = {
        "preferred_detail_level": "technical" if profile.get("department") == "Engineering" else "business",
        "preferred_doc_types": [profile.get("department", "General")],
        "query_patterns": profile.get("recent_topics", [])
    }
    
    return {
        "user_profile": profile,
        "conversation_history": history,
        "learned_preferences": learned_prefs
    }

print("✅ load_user_context function defined")

In [ ]:
# Dual Retrieval Node

def dual_retrieval(state: MemoryRAGState) -> dict:
    """
    Retrieve from both static knowledge base AND user memory.
    
    Context Engineering Pillar: SELECT
    - Traditional RAG: Only static docs
    - Memory-augmented RAG: Static + conversation + user context
    - 45% improvement in relevance
    """
    
    print(f"\n{'='*70}")
    print(f"🔍 DUAL RETRIEVAL")
    print(f"{'='*70}\n")
    
    question = state["question"]
    user_profile = state["user_profile"]
    conversation_history = state["conversation_history"]
    
    # 1. STATIC DOCUMENT RETRIEVAL
    print("📚 Searching static knowledge base...")
    static_results = vectorstore.similarity_search(question, k=3)
    
    retrieved_static = [
        {
            "content": doc.page_content.strip(),
            "metadata": doc.metadata
        }
        for doc in static_results
    ]
    
    print(f"   Found {len(retrieved_static)} relevant documents:")
    for i, doc in enumerate(retrieved_static, 1):
        print(f"   {i}. {doc['metadata'].get('title', 'Untitled')} ({doc['metadata'].get('category', 'General')})")
    
    # 2. CONVERSATION MEMORY RETRIEVAL
    print(f"\n💭 Searching conversation history...")
    question_lower = question.lower()
    relevant_history = [
        conv for conv in conversation_history
        if any(keyword in conv["question"].lower() for keyword in question_lower.split())
    ]
    
    print(f"   Found {len(relevant_history)} relevant past conversations")
    
    # 3. USER PROFILE FACTS
    print(f"\n👤 Extracting relevant user facts...")
    user_facts = []
    if user_profile.get("department"):
        user_facts.append(f"User works in {user_profile['department']} department")
    if user_profile.get("role"):
        user_facts.append(f"User role: {user_profile['role']}")
    if user_profile.get("interests"):
        user_facts.append(f"User interests: {', '.join(user_profile['interests'])}")
    
    print(f"   • {len(user_facts)} contextual facts extracted")
    
    return {
        "retrieved_static_docs": retrieved_static,
        "retrieved_memory_context": relevant_history,
        "relevant_user_facts": user_facts
    }

print("✅ dual_retrieval function defined")

In [ ]:
# Contextualized Generation Node

def contextualized_generation(state: MemoryRAGState) -> dict:
    """
    Generate answer using ALL available context.
    
    Combines: Static knowledge + Conversation memory + User context
    """
    
    print(f"\n{'='*70}")
    print(f"✨ GENERATING CONTEXTUALIZED ANSWER")
    print(f"{'='*70}\n")
    
    llm = get_chat_model()  # Uses DIAL
    
    # Build comprehensive prompt
    static_docs_text = "\n\n".join([
        f"**{doc['metadata'].get('title', 'Untitled')}**\n{doc['content']}"
        for doc in state["retrieved_static_docs"]
    ])
    
    history_text = "\n".join([
        f"Q: {conv['question']}\nA: {conv['answer']}"
        for conv in state["retrieved_memory_context"]
    ]) if state["retrieved_memory_context"] else "No previous conversations on this topic"
    
    user_context_text = "\n".join([f"- {fact}" for fact in state["relevant_user_facts"]])
    
    prompt = f"""You are a helpful enterprise knowledge assistant. Provide a personalized, context-aware answer.

**STATIC KNOWLEDGE BASE:**
{static_docs_text}

**PREVIOUS CONVERSATIONS:**
{history_text}

**USER CONTEXT:**
{user_context_text}

**CURRENT QUESTION:**
{state['question']}

**INSTRUCTIONS:**
1. Answer the question using the knowledge base documents
2. Consider the user's department and role when framing your response
3. Reference previous conversations if relevant
4. Be specific and cite sources
5. Adapt detail level to user's technical background

**ANSWER:**"""
    
    response = llm.invoke(prompt)
    answer = response.content.strip()
    
    # Track sources used
    sources = [doc["metadata"].get("title", "Untitled") for doc in state["retrieved_static_docs"]]
    
    print(f"📝 Generated Answer:")
    print(f"{answer}\n")
    print(f"📖 Sources: {', '.join(sources)}")
    
    return {
        "answer": answer,
        "sources_used": sources
    }

print("✅ contextualized_generation function defined")

In [ ]:
# Update Memory Node

def update_memory(state: MemoryRAGState) -> dict:
    """
    Extract new facts and update user memory.
    
    Context Engineering Pillar: WRITE (to external memory)
    """
    
    print(f"\n{'='*70}")
    print(f"💾 UPDATING USER MEMORY")
    print(f"{'='*70}\n")
    
    # Add this Q&A to conversation history
    new_conversation = {
        "question": state["question"],
        "answer": state["answer"],
        "timestamp": datetime.now().isoformat()
    }
    
    # Extract key topics
    question_keywords = [word.lower() for word in state["question"].split() if len(word) > 4]
    
    # Update user profile interests
    existing_interests = set(state["user_profile"].get("interests", []))
    updated_interests = list(existing_interests.union(set(question_keywords[:3])))
    
    memory_updates = {
        "new_conversation": new_conversation,
        "updated_interests": updated_interests,
        "last_interaction": datetime.now().isoformat()
    }
    
    print(f"✅ Memory updated:")
    print(f"   • Conversation saved")
    print(f"   • Last interaction: {memory_updates['last_interaction']}")
    
    return {"memory_updates": memory_updates}

print("✅ update_memory function defined")

In [ ]:
# Build and Compile the Graph

# Build the memory-augmented RAG workflow
workflow = StateGraph(MemoryRAGState)

# Add nodes
workflow.add_node("load_context", load_user_context)
workflow.add_node("dual_retrieval", dual_retrieval)
workflow.add_node("generate", contextualized_generation)
workflow.add_node("update_memory", update_memory)

# Add edges
workflow.add_edge(START, "load_context")
workflow.add_edge("load_context", "dual_retrieval")
workflow.add_edge("dual_retrieval", "generate")
workflow.add_edge("generate", "update_memory")
workflow.add_edge("update_memory", END)

# Compile
app = workflow.compile()

print("✅ Memory-Augmented RAG workflow compiled")
print("\nWorkflow Structure:")
print("   START → load_context → dual_retrieval → generate → update_memory → END")

In [ ]:
# Run Example: Engineer Query

print("\n" + "="*70)
print("EXAMPLE: Engineer - API Authentication Query")
print("="*70)

result = app.invoke({
    "user_id": "user_eng_001",
    "question": "How long do API tokens last and how do I refresh them?"
})

print("\n📝 FINAL ANSWER:")
print(result["answer"])
print(f"\n📖 Sources: {', '.join(result['sources_used'])}")
print("\n" + "="*70)

In [ ]:
# Key Learnings

print("""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 TRADITIONAL RAG VS MEMORY-AUGMENTED RAG
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

TRADITIONAL RAG:
  Query → Vector Search → Generate
  ❌ Stateless (no session memory)
  ❌ No user context
  ❌ Same answer for everyone
  ✅ Simple implementation

MEMORY-AUGMENTED RAG:
  Query → Dual Retrieval (Static + Memory + Profile) → Generate → Update Memory
  ✅ Stateful across sessions
  ✅ User-aware and personalized
  ✅ Learns from interactions
  ✅ 45% relevance improvement
  ❌ More complex architecture

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📚 PRODUCTION LEARNINGS FROM NOTION AI
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1️⃣  DUAL RETRIEVAL ARCHITECTURE
   • Static docs: 60% of context
   • Conversation memory: 25% of context
   • User profile: 15% of context

2️⃣  MEMORY MANAGEMENT
   • Short-term: Last 5-10 conversations (Redis, 24hr TTL)
   • Long-term: User profile & preferences (PostgreSQL)
   • Episodic: Past interactions (vector store)

3️⃣  PERSONALIZATION STRATEGIES
   • Department-aware document filtering
   • Role-based detail level (technical vs business)
   • Interest-based ranking boost

4️⃣  ACCURACY METRICS
   • Relevance: 45% improvement vs traditional RAG
   • Repeated questions: 60% reduction
   • User satisfaction: 3.2 → 4.5/5 stars

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
""")

---

## Best Practices Summary

### When to Use Each Pillar

**Use WRITE when:**
- Conversations exceed 10+ turns
- Agent needs to remember facts across sessions
- Task requires step-by-step planning

**Use SELECT when:**
- You have 30+ tools (use tool RAG)
- You have 100+ documents (use knowledge RAG)
- You have historical conversations (use memory RAG)

**Use COMPRESS when:**
- Context approaches model's limit (e.g., 100K tokens)
- Tool outputs are verbose (e.g., full file contents)
- Conversation history is repetitive

**Use ISOLATE when:**
- Task requires multiple specialized skills
- Risk of context clash between subtasks
- Code execution needs sandboxing

---

## Key Takeaways

### Context Engineering Framework

| Pillar | Action | LangGraph Feature | When to Use |
|--------|--------|-------------------|-------------|
| **WRITE** | Save outside context | `InMemoryStore`, `PostgresStore` | Long conversations, cross-session memory |
| **SELECT** | Pull in when needed | `store.search()`, RAG retrieval | 30+ tools, 100+ documents |
| **COMPRESS** | Keep only relevant | `SummarizationMiddleware`, reducers | Context approaching limit |
| **ISOLATE** | Split up context | Subgraphs, multi-agent | Specialized tasks, code execution |


---

## 🎉 Session 2 Complete!

### What You've Learned

| Module | Topic | Key Skills |
|--------|-------|------------|
| **Module 1** | Memory & Streaming | Store API, semantic search, token streaming |
| **Module 2** | HITL & Durability | `interrupt()`, `Command(resume=...)`, `@task` |
| **Module 3** | Self-Reflection | Quality gates, LLM-as-Judge, critique loops |
| **Module 4** | Multi-Agent Collaboration | Peer review, Send API, handoffs |
| **Module 5** | Context Engineering | WRITE/SELECT/COMPRESS/ISOLATE, middleware |

### Continue to Session 3

- **Module 1**: Evaluation frameworks (automated testing)
- **Module 2**: Observability and cost optimization
- **Module 3**: Security and guardrails
- **Module 4**: Production case studies (Klarna, Replit, Elastic)
- **Module 5**: MCP integration for tool interoperability